# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('IDataset2.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Explosive type     588 non-null    object 
 1   Explosive mass     588 non-null    float64
 2   Standoff distance  588 non-null    float64
 3   Impulse            588 non-null    float64
dtypes: float64(3), object(1)
memory usage: 18.5+ KB


In [3]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Explosive type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Explosive mass                588 non-null    float64
 1   Standoff distance             588 non-null    float64
 2   Impulse                       588 non-null    float64
 3   Explosive type_Composition B  588 non-null    uint8  
 4   Explosive type_TNT            588 non-null    uint8  
dtypes: float64(3), uint8(2)
memory usage: 15.1 KB


In [4]:
dataset.head()

,Explosive mass,Standoff distance,Impulse,Explosive type_Composition B,Explosive type_TNT
0,0.5,1.0,77.067900,0,1
1,0.5,1.5,58.168617,0,1
2,0.5,2.5,38.798874,0,1
3,0.5,3.5,28.473915,0,1
4,0.5,4.5,22.343452,0,1


In [5]:
y = dataset[('Impulse')]
X = dataset.drop('Impulse', axis=1)
print(X.shape, y.shape)

(588, 4) (588,)


In [6]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 71)

In [8]:
X_val, X_test, y_val, y_test = train_test_split(X_test,
                                                y_test,
                                                test_size = 0.5,
                                                random_state = 71)

### Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,0:2] = sc.fit_transform(X_train[:, 0:2])
print (X_train)

In [ ]:
X_test[:,0:2] = sc.transform(X_test[:, 0:2])
print (X_test)

In [ ]:
X_val[:,0:2] = sc.transform(X_val[:, 0:2])
print (X_val)

### S1 - Hyperparameter tuning - layers, neurons, activation function

In [13]:
import math
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam


# Set seed for NumPy
np.random.seed(71)

# Set seed for TensorFlow
tf.random.set_seed(71)

def FindLayerNodesLinear( last_layer_nodes):
    layers = []
    nodes_increment = (last_layer_nodes - 20)/ 2
    nodes = 20
    for i in range(1, 4):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [14]:
def create_model1(last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(last_layer_nodes)
    for i in range(1, 4):
        if i==1:
            model.add(Dense(units = 20,  input_shape=(X_train.shape[1],), activation=activation_func))
            model.add(Dropout(0.1))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            model.add(Dropout(0.1))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    optimizer1 = Adam(learning_rate=0.01)
    model.compile(optimizer = optimizer1, loss = 'mean_squared_error', metrics = ['mae'])
    return model

activation_func = ['relu', 'leaky_relu', 'softplus']
last_layer_nodes = [10, 20, 30, 40, 50,60, 70, 80,90,100,110,120,130, 140, 150,160, 170, 180, 190, 200]

param_grid = dict(model__activation_func = activation_func,model__last_layer_nodes = last_layer_nodes)

##Wrap model into scikit-learn
model1 = KerasRegressor(model=create_model1, verbose=0, epochs = 100, batch_size = 50, random_state = 71)

In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state=71)
grid1 = GridSearchCV(estimator = model1, param_grid= param_grid, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result1 = grid1.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result1.best_score_, grid_result1.best_params_))
means = grid_result1.cv_results_['mean_test_score']
stds = grid_result1.cv_results_['std_test_score']
params = grid_result1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

D:\Anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.971237 using {'model__activation_func': 'relu', 'model__last_layer_nodes': 200}
0.956167 (0.013286) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 10}
0.960763 (0.008021) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 20}
0.964636 (0.005536) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 30}
0.966952 (0.007781) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 40}
0.963682 (0.010396) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 50}
0.967761 (0.007490) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 60}
0.964836 (0.010330) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 70}
0.964245 (0.010193) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 80}
0.964672 (0.013967) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 90}
0.961957 (0.021670) with: {'model__activation_func': 'relu', 'model__last_layer_node

In [12]:
pd.concat([pd.DataFrame(params), pd.DataFrame({'model__first_layer_nodes': [20] * len(params)}),pd.DataFrame(means, columns=['R2'])], axis =1)

,model__activation_func,model__last_layer_nodes,model__first_layer_nodes,R2
0,relu,10,10,0.957277
1,relu,20,10,0.952770
2,relu,30,10,0.955750
3,relu,40,10,0.962781
4,relu,50,10,0.963277
5,relu,60,10,0.961489
6,relu,70,10,0.961639
7,relu,80,10,0.961165
8,relu,90,10,0.967092
9,relu,100,10,0.957607


### Merge all files

In [ ]:
from pathlib import Path

#Access input folder
input_dir = Path ("Pressure_hyperparameter_tuning_3layers")
print ("1",input_dir)

# Output Excel file
output_excel_file = Path("Pressure_hyperparameter_tuning_3layers/S1_summary_3layers.xlsx")

# List to store DataFrames from CSV files
dfs = []

# Loop through CSV files in the directory
for csv_file in input_dir.glob('*.csv'):
    # Read CSV file into a DataFrame and append to the list
    df = pd.read_csv(csv_file)
    dfs.append(df)

# Concatenate DataFrames in the list along rows
merged_df = pd.concat(dfs, ignore_index=True)

# Write the merged DataFrame to an Excel file
merged_df.to_excel(output_excel_file, index=False)

In [ ]:
# Set seed for NumPy
np.random.seed(71)

# Set seed for TensorFlow
tf.random.set_seed(71)

# Function to create model, required for KerasClassifier
def create_model2():
 # create model
 model = Sequential()
 model.add(Dense(units=40, input_shape=(X_train.shape[1],), activation='relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=85, activation='relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=130, activation='relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=1, activation='linear'))

 return model

In [ ]:
# create model
model2 = KerasRegressor(model=create_model2, verbose=0, random_state = 71, loss = 'mean_squared_error', metrics = ['mae'])

# define the grid search parameters


batch_size = [30,40,50]
optimizer = [Adam, Nadam, RMSprop]
learning_rate = [ 0.001,0.01, 0.1]
epochs = [100, 200, 300, 400, 500]

# gridsearch
param_grid2 = dict(batch_size=batch_size, optimizer=optimizer, optimizer__learning_rate = learning_rate, epochs = epochs)
kf = KFold(n_splits=5, shuffle=True, random_state=71)
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid2, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result2 = grid2.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))